In [2]:
import sympy as sp
import numpy as np
import math

#### variables globales
1. variables : une liste contient des symboles créés; utilisé dans la partie expressions
2. 
3.

## Step 0 : dénifir la fonction f() = (N-pq)²

In [3]:
# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

### Définir variables pour p et q
##### p en binaire peut être représenté comme:
##### (pl1-1,..p1,1) = pl1-1 * 2 ** (l1-1) + ... + p1 * 2 **1 +1
##### q en binaire peut être représenté comme:
##### (ql2-1,..,q1,1) = ql2-1 * 2 ** (l2-1) + ... + q1 * 2 **1 +1

In [4]:
def set_variable(l1,l2):
    #p
    expr_p = 1
    for i in range(1,l1):
        var_name = ('p'+ str(i))
        var = create_variables(var_name)
        expr_p = 2**i * var + expr_p 
    #q
    expr_q = 1
    for i in range(1,l2):
        var_name = ('q'+ str(i))
        var = create_variables(var_name)
        expr_q = 2**i * var + expr_q
    return (expr_p,expr_q)

### Fonction f = (N-p*q)²
##### Le min de cette fonction dans Z est(N-p*q)²=0, dans ce cas N = p*q.

In [5]:
def define_function(N,l1,l2):
    expr_pq = 1
    pq_variable = set_variable(l1,l2)
    for item in pq_variable:
        expr_pq = expr_pq * item
    return (N - expr_pq)**2

#expr = define_function(15)
#expr


### Step 1: Réduire la fonction en mettant le power des variables en 1
##### Par exemple de p_1**2 à p_1, car 1 * 1 = 1 & 0 * 0 = 0

In [6]:
def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

#expr_reduct_power = reduct_variable_power_function(expr)
#expr_reduct_power


## Step 2: Réduire les termes qui ont une degrée max est 2.

In [7]:
# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number 
    letter = name[0]
    number = int(name[1:])
    return (letter, number)


### réduction degrée formule

In [8]:

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term


# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):    
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]
    
    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1 
    else:
        x_4 = key 

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)


### Réduction procède

In [9]:
def reduct_demension_function(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :  
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste) 
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_function(new_expr)
    else:
        return new_expr

#reduct_demension_expr = reduct_demension_function(expr_reduct_power)
#reduct_demension_expr

## Step3: Calculer h et J

1. Ising : transfer de Qubo à Ising 
###### De reduct_demension_expr, on utilise les formule comme:
###### a * x_i = a * (s_i + 1) / 2 
###### (a * x_i * x_j) = a * (s_i + 1) * (s_j + 1) / 4

In [10]:
# créer variable à partir des variables dans modèle d'Ising

def create_variable_Ising(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables_Ising.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol


# définir H
def transfers_qubo2Ising(reduct_demension_expr):

    n = len(variables)
    H = 0
    for i in range(1,n+1):
        var_name = ('s'+ str(i))
        create_variable_Ising(var_name)

    # variable = (variable_ising +1)/2
    subs_relations = [(variables[i], (variables_Ising[i] + 1) / 2) for i in range(n)]
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr


In [11]:
def build_h_Ising(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h = h + (term.as_coeff_mul()[0]/-2 ,)
    return h

def build_J_h_Ising(tranfrers_expr):    
    terms = tranfrers_expr.args 
    # Update le coef des termes avec seulement des termes avec dimension 1.
    new_expr = 0
    # Matrice contient les coef des termes avec dimension 2. 
    matrix_J = np.zeros((len(variables_Ising), len(variables_Ising)), dtype=int)
    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]#/4
            i = variables_Ising.index(list_variable_term[0])
            j = variables_Ising.index(list_variable_term[1])
            matrix_J[i][j] = coef/2
        else:
            # 1-variable : reconstruire une expression pour calculer le coef des termes
            new_expr += term
    tuple_h = build_h_Ising(new_expr)

    return tuple_h,matrix_J

2. Qubo : on utilise directement l'expression reduct_demension_expr pour calculer h et J

In [12]:
def build_h_Qubo(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h += (term.as_coeff_mul()[0],)
    return h

def build_J_h_Qubo(reduct_demension_expr):
    terms = reduct_demension_expr.args 
    new_expr = 0
    matrix_J = np.zeros((len(variables), len(variables)), dtype=int)
    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]
            i = variables.index(list_variable_term[0])
            j = variables.index(list_variable_term[1])
            matrix_J[i][j] = coef
        else:
            # 1-variable : reconstruire une expression pour calculer le coef des termes
            new_expr += term
    tuple_h = build_h_Qubo(new_expr)
    return tuple_h,matrix_J

sum up fonction

In [13]:
def get_reduct_expression(N,l1,l2):
    expr = define_function(N,l1,l2)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_function(expr_reduct_power)
    return reduct_demension_expr

def get_h_and_J_Ising(N,l1,l2):
    reduct_demension_expr = get_reduct_expression(N,l1,l2)
    ising_expr = transfers_qubo2Ising(reduct_demension_expr)
    tuple_h,matrix_J = build_J_h_Ising(ising_expr)
    return tuple_h,matrix_J

def get_h_and_J_Qubo(N,l1,l2):
    reduct_demension_expr = get_reduct_expression(N,l1,l2)
    tuple_h,matrix_J = build_J_h_Qubo(reduct_demension_expr)
    return tuple_h,matrix_J

## Step4: Solver pour trouver la solution

In [14]:
#import qubo modèle huristique
from pyqubo import Spin,Binary
from pyqubo import solve_qubo
import neal

### Définir la formule Hamiltonian

1. Ising

In [15]:
# définir formule H
def H_pyqubo_ising(N,l1,l2):
    (h,J)=get_h_and_J_Ising(N,l1,l2)
    print("****************")
    print(f"N est : {N}")
    print(f"h est : {h}")
    print(f"J est : {J}")

    hamiltonian_expr = 0
    n=len(h)

    # définir spin; nombre de spin == nombre de variables
    spins = [Spin(f's{i + 1}') for i in range(n)]
    for i in range(n):
        for j in range(n):
            hamiltonian_expr -= J[i, j] * spins[i] * spins[j]
        hamiltonian_expr -= h[i] * spins[i]
    return hamiltonian_expr


2. Qubo

In [16]:
def H_pyqubo_qubo(N,l1,l2):
    (h,J)=get_h_and_J_Qubo(N,l1,l2)
    print("****************")
    print(f"N est : {N}")
    print(f"h est : {h}")
    print(f"J est : {J}")

    hamiltonian_expr = 0
    n=len(h)

    # définir qubo binaire; nombre qubo binaire == nombre de variables
    #binary = [Binary(f'b{i + 1}') for i in range(n)]

    binary = []
    #dict_variable_binary={}
    num = 1

    for i in variables:
        bin = Binary(f'b{num}')
        binary.append(bin)
        dict_variable_binary[i]= bin
        num += 1

    for i in range(n):
        for j in range(n):
            hamiltonian_expr -= J[i, j] * binary[i] * binary[j]
        hamiltonian_expr -= h[i] * binary[i]

    return hamiltonian_expr

### Compiler pour obtenir modèle

In [17]:
def model_qubo(hamiltonian_expr):
    model = hamiltonian_expr.compile()  
    qubo, offset = model.to_qubo()
    solution = solve_qubo(qubo, num_reads=10)
    return solution

In [18]:
def model_qubo1(hamiltonian_expr):
    model = hamiltonian_expr.compile()  
    bqm = model.to_bqm()
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=10)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    return best_sample.sample

# Call function

In [19]:
def factorization_I(N,l1,l2):
    hamiltonian_expr_i = H_pyqubo_ising(N,l1,l2)
    solution = model_qubo1(hamiltonian_expr_i)
    print(solution)
    print_p_q_I(solution,N)
    return solution

def print_p_q_I(solution,N):
    p_bits = []
    q_bits = []

    for varIsing, var in zip(variables_Ising, variables):
        if var.name.startswith('p'):
            p_bits.append(solution[varIsing.name])
        elif var.name.startswith('q'):
            q_bits.append(solution[varIsing.name])

    p_bits.append(1)
    q_bits.append(1) 

    p_bin_str = ''.join(map(str, p_bits))
    q_bin_str = ''.join(map(str, q_bits))

    p_dec = int(p_bin_str, 2)
    q_dec = int(q_bin_str, 2)

    print(f"La factorisation de {N} est {p_dec} * {q_dec}.")

In [24]:
def factorization_Q(N,l1,l2):
    hamiltonian_expr_q = H_pyqubo_qubo(N,l1,l2)
    solution = model_qubo1(hamiltonian_expr_q)
    return solution

def print_p_q_Q(solution,N):
    p_bits = []
    q_bits = []

    #for varIsing, var in zip(variables, variables):
    #    if var.name.startswith('p'):
    #        p_bits.append(solution[varIsing.name])
    #    elif var.name.startswith('q'):
    #        q_bits.append(solution[varIsing.name])

    p_bits.append(1)
    q_bits.append(1) 

    p_bin_str = ''.join(map(str, p_bits))
    q_bin_str = ''.join(map(str, q_bits))

    p_dec = int(p_bin_str, 2)
    q_dec = int(q_bin_str, 2)

    print(f"La factorisation de {N} est {p_dec} * {q_dec}.")

    return p_dec * q_dec == N

In [25]:
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_binary={}
#variales_Ising: une liste global contient des string qui sont le nom des variables dans qubo, sert à compter le nombre de variables dans qubo
N=25
l1=3
l2=3
solution = factorization_I(N,l1,l2)

solution1 = factorization_Q(N,l1,l2)

****************
N est : 25
h est : (640, 640, 240, 192, 96, -80, -160, -240, -608, -720)
J est : [[   0  258   15 -282 -512  -64    0  512    0    0]
 [   0    0   70 -284 -512    0 -192    0  512    0]
 [   0    0    0   66   16  -64 -192    0    0 -128]
 [   0    0    0    0  304   16   48  512  512 -128]
 [   0    0    0    0    0    0    0 -128 -128   32]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]
{'s1': 1, 's3': 1, 's10': 0, 's2': 1, 's4': 0, 's5': 0, 's6': 0, 's7': 0, 's8': 0, 's9': 0}
La factorisation de 25 est 7 * 5.
****************
N est : 25
h est : (-6144, -6144, -176, -176, -92, -92, 768, 1536, 2304, 6144)
J est : [[    0  2064   120 -2256 -4096  -512     0  4096     0     0     0     0
      0     0]
 [    0     0   560 -2272 -4096     0 -1536     0  4

In [22]:
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_binary={}
#variales_Ising: une liste global contient des string qui sont le nom des variables dans qubo, sert à compter le nombre de variables dans qubo
N=35
l1=3
l2=3
solution = factorization_I(N,l1,l2)

****************
N est : 35
h est : (640, 640, 280, 272, 136, -80, -160, -240, -608, -640)
J est : [[   0  258    5 -302 -512  -64    0  512    0    0]
 [   0    0   50 -324 -512    0 -192    0  512    0]
 [   0    0    0   66   16  -64 -192    0    0 -128]
 [   0    0    0    0  304   16   48  512  512 -128]
 [   0    0    0    0    0    0    0 -128 -128   32]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]
{'s1': 1, 's3': 1, 's10': 0, 's2': 1, 's4': 0, 's5': 0, 's6': 0, 's7': 0, 's8': 0, 's9': 0}
La factorisation de 35 est 7 * 5.


## Solver2 DKenefake/Hercules


In [23]:
import hercules
import random
import time

# set a random seed
random.seed(time.time())

# read in the qubo problem
problem = hercules.read_qubo('test_large.qubo')
num_x = problem[-1]

# create a random initial point
x_0 = [random.randint(0, 1) for i in range(num_x)]

# start timing
start = time.time()

# solve the QUBO problem via the mixed local search heuristic with, initial point x_0, for 50 iterations
x_soln, obj = hercules.mls(problem, x_0, 50)

# stop timing
end = time.time()

# print the objective value of the solution
print('Solution: ', obj, ' in ', end - start, ' seconds')


ModuleNotFoundError: No module named 'hercules'